### Using GANs to Generate Handwritten Digits

#### Import Relevant Packages and Define the Device

In [12]:
from torch_snippets import *
from torchvision.datasets import MNIST
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torch import nn, optim
from torchsummary import summary
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Import the Dataset and Define the Dataloader

In [13]:
transform = T.Compose([
                       T.ToTensor(),
                       T.Normalize((0.5,), (0.5,))
])

In [14]:
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)

In [15]:
train_data_dl = DataLoader(train_dataset, batch_size=128, shuffle=True , drop_last=True)

#### Define the Discriminator Model Class

In [20]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
                                    nn.Linear(784, 1024), 
                                    nn.LeakyReLU(0.2),
                                    nn.Dropout(0.3),
                                    nn.Linear(1024, 512),
                                    nn.LeakyReLU(0.2),
                                    nn.Dropout(0.3),
                                    nn.Linear(512, 256),
                                    nn.LeakyReLU(0.2),
                                    nn.Dropout(0.3),
                                    nn.Linear(256, 1),
                                    nn.Sigmoid()
                                  )
    
    def forward(self, x):
        x = self.model(x)
        return x

In [21]:
discriminator = Discriminator().to(device)
summary(discriminator, (1,28*28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1024]         803,840
         LeakyReLU-2              [-1, 1, 1024]               0
           Dropout-3              [-1, 1, 1024]               0
            Linear-4               [-1, 1, 512]         524,800
         LeakyReLU-5               [-1, 1, 512]               0
           Dropout-6               [-1, 1, 512]               0
            Linear-7               [-1, 1, 256]         131,328
         LeakyReLU-8               [-1, 1, 256]               0
           Dropout-9               [-1, 1, 256]               0
           Linear-10                 [-1, 1, 1]             257
          Sigmoid-11                 [-1, 1, 1]               0
Total params: 1,460,225
Trainable params: 1,460,225
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

#### Define the Generator Model Class

In [22]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
                                   nn.Linear(100, 256),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(256, 512),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(512, 1024),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(1024, 784),
                                   nn.Tanh()
                                    )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [23]:
generator  = Generator().to(device)
summary(generator, (1,100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]          25,856
         LeakyReLU-2               [-1, 1, 256]               0
            Linear-3               [-1, 1, 512]         131,584
         LeakyReLU-4               [-1, 1, 512]               0
            Linear-5              [-1, 1, 1024]         525,312
         LeakyReLU-6              [-1, 1, 1024]               0
            Linear-7               [-1, 1, 784]         803,600
              Tanh-8               [-1, 1, 784]               0
Total params: 1,486,352
Trainable params: 1,486,352
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 5.67
Estimated Total Size (MB): 5.71
----------------------------------------------------------------
